# Tool Calling: Extending What Agents Can Do

In the previous chapter, you built AI agents that used tools like Calculator and Wikipedia. The agent decided *when* to call each tool based on the user's question. This chapter goes deeper: how do tools actually work, how do you design them well, and how do you keep agents safe?

**What you'll learn:**
- How tool calling works under the hood
- The critical difference between read-only and write tools
- How to write tool descriptions that actually work
- Guardrails to prevent agents from causing problems
- How to build a complete research-to-action workflow with approval

---

---

## How Tool Calling Works

When you connect tools to an AI Agent node, you're giving the model a menu of capabilities. The model doesn't execute tools directly—it *requests* that a tool be called, and n8n executes it.

### The Loop

This is the same agent loop from the First AI Agent chapter: **Reason → Call Tool → Observe → Repeat or Answer** (also called the **ReAct pattern**).

1. **Reason** — The model reads the user's question and its available tools
2. **Call Tool** — The model outputs which tool to call and with what inputs
3. **Observe** — n8n executes the tool and returns the result to the model
4. **Repeat or Answer** — The model decides if it needs more tools or can answer

### Why It Matters

The model chooses tools based on:
- The **tool name** and **description** you provide
- The **user's question**
- The **system message** instructions

If any of these are unclear, the model may call the wrong tool, call no tool, or hallucinate an answer instead of looking it up.

---

## How the Agent Picks a Tool

When you connect tools to an AI Agent, n8n collects all their descriptions and sends them to the LLM. The model then decides which tool to use—you don't write "if user asks X, use tool Y" logic.

**What the model uses to decide:**

- **Tool descriptions** — What each tool does (the main signal)
- **Tool names** — Short identifiers like `wikipedia`, `calculator`
- **User's question** — What they're asking for
- **System Message** — Extra rules you add (optional)

### What a Tool Looks Like

Every tool has a name, a description, inputs, and outputs:

| Part | Example (Wikipedia) |
|------|---------------------|
| **Name** | `wikipedia` |
| **Description** | "Search Wikipedia for factual information" |
| **Inputs** | `query` (string) |
| **Outputs** | Article text or summary |

The **description** is the most important part—it tells the model when to use the tool.

### Where Descriptions Come From

| Tool type | Description | You write it? |
|-----------|-------------|---------------|
| Built-in (Wikipedia, Calculator) | Hardcoded in the node | No |
| Custom (HTTP Request Tool, Workflow Tool) | Field in the tool's settings | Yes |

```{tip}
For built-in tools, the descriptions are hidden in the code. n8n is open source, so you can see them: [n8n tool source code](https://github.com/n8n-io/n8n/tree/master/packages/%40n8n/nodes-langchain/nodes/tools).

Wikipedia's description is: *"A tool for interacting with and fetching data from the Wikipedia API."*
```

### The System Message: Extra Rules, Not Descriptions

The System Message is for *additional rules*, not for describing tools. Use it when:

- You want to prioritize one tool: *"Always try Wikipedia first"*
- You need to limit usage: *"Call each tool at most twice"*
- You want to prevent actions: *"Never send emails without asking"*

If your tools have clear descriptions, the agent will usually pick correctly without extra help.

---

## Read-Only vs Write Tools

This is the most important distinction in tool design:

| Type | What it does | Risk | Default policy |
|------|--------------|------|----------------|
| **Read-only** | Fetches or calculates data | Low | Allow freely |
| **Write** | Changes something in the real world | High | Require approval |

### Read-Only Tools (Safe)

- Wikipedia lookup
- Calculator
- Weather API
- Database query (SELECT only)
- Web search

These tools only *read* information. If the agent calls them incorrectly, nothing bad happens—you just get wrong or irrelevant data.

### Write Tools (Dangerous)

- Send email
- Create ticket
- Update database
- Post to social media
- Delete file

These tools *change* something. If the agent calls them incorrectly, you could send embarrassing emails, create duplicate tickets, or lose data.

### The Golden Rule

> **If a tool changes something in the real world, it needs human approval before executing.**

### Principle of Least Privilege

Give agents only the tools they need:
- A research assistant only needs read-only tools
- A customer support agent might need write tools, but with approval gates

Start minimal. Add more tools only when necessary.

```{note}
**What if you have many tools?**

This course uses 2-3 tools per agent. In production systems with 10+ tools, new challenges appear: the agent may pick the wrong tool, calls become slow and expensive, or tools from different domains interfere with each other.

Common solutions:
- **Tool Search**: Retrieve only relevant tools per request (instead of loading all 50)
- **Programmatic Tool Calling**: Agent writes code that calls multiple tools in batch, returns only the result
- **Universal Code Runner**: One powerful Code node instead of many small tools (n8n's Code node is exactly this)
- **Router → Specialists**: Split into multiple focused agents (e.g., Support Agent, Billing Agent) with isolated tool sets
- **Tool Use Examples**: Add example calls inside tool descriptions to improve parameter accuracy

For implementation details, see [Anthropic's Advanced Tool Use](https://www.anthropic.com/engineering/advanced-tool-use).
```

---

## Writing Good Descriptions

For custom tools (HTTP Request Tool, Workflow Tool), you write the description in the **tool node's Description field**—not in the System Message.

A good description tells the model *when* to use the tool and *when not to*:

```
Use this tool when [specific situation].
Do NOT use when [counter-situation].
Input: [what it needs]
Output: [what it returns]
```

### Examples

**Bad description:**
```
Searches Wikipedia.
```

**Good description (in the tool's Description field):**
```
Use when the user asks about facts: people, places, historical events.
Do NOT use for opinions or recent news.
Input: search query (string)
Output: article summary
```

**Bad description:**
```
Sends an email.
```

**Good description:**
```
Use ONLY when user explicitly asks to send a message.
Do NOT use to "draft" or "prepare" — ask first if unsure.
Input: recipient, subject, body
WARNING: Cannot be undone.
```

### Test Your Descriptions

If the agent picks the wrong tool, improve the description—don't add complex logic to the System Message.

| Prompt | Expected tool |
|--------|---------------|
| "Who invented the telephone?" | Wikipedia |
| "What's 15% of 340?" | Calculator |
| "Send a thank-you email to John" | Send Email |
| "Help me write an email" | Draft Email (not Send) |

---

## Guardrails

Even with good descriptions, agents can misbehave. The Guardrails & Safety chapter covers this topic in depth. Here are the essentials:

- **Max Iterations:** Set to 5-10 in AI Agent → Settings (gear icon). Prevents infinite loops.
- **System Message rules:** Add explicit limits like "Call Wikipedia at most twice per question" and "If a tool fails, explain the error and stop."
- **Approval gates:** For write tools (send email, create ticket), add a **Wait node** before the action so a human reviews the output first.

### Principle of Least Privilege

Give agents only the tools they need:
- A research assistant only needs read-only tools
- A customer support agent might need write tools, but with approval gates

Start minimal. Add more tools only when necessary.

---

## Putting It Together: Research Agent with Approval

This workflow demonstrates everything we've learned:
- An agent with **read-only tools** (Wikipedia, Calculator)
- An **approval gate** before any write action could happen
- Good **tool descriptions** and **guardrails**

We use familiar tools (Wikipedia, Calculator) so you can focus on the new concepts: safety patterns, tool contracts, and approval gates.

```
┌──────────────────┐     ┌──────────────────┐     ┌──────────────────┐
│  Manual Trigger  │────▶│  Input: Topic    │────▶│   AI Agent       │
└──────────────────┘     └──────────────────┘     │  (Research)      │
                                                  └──────────────────┘
                                                          │
                                          ┌───────────────┴───────────────┐
                                          │                               │
                                    [Wikipedia]                    [Calculator]
                                    (read-only)                    (read-only)
                                          │
                                          ▼
                                ┌──────────────────┐
                                │  Format Summary  │
                                │  + Draft Email   │
                                └──────────────────┘
                                          │
                                          ▼
                                ┌──────────────────┐
                                │  ⏸️ WAIT for    │
                                │    Approval     │
                                └──────────────────┘
                                          │
                                    Human approves
                                          │
                                          ▼
                                ┌──────────────────┐
                                │  Output: Ready   │
                                │  to Send         │
                                └──────────────────┘
```

### The Mission

1. User provides a research topic
2. Agent looks up information using Wikipedia
3. Agent calculates any numbers if needed
4. Workflow formats findings as an email draft
5. **Human reviews and approves** the draft

**Note:** This workflow stops at approval. In production, you would connect a Gmail/Outlook node after the Wait to actually send the email. The pattern is: AI drafts → human approves → system sends.

---

### Import the Workflow

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/09_tool_calling_research_approval.json
> ```
>
> **Download:** {download}`09_tool_calling_research_approval.json <_static/workflows/09_tool_calling_research_approval.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create the workflow

1. Click **Workflows** → **Add Workflow**
2. Rename to "Research Agent with Approval"

### Step 2: Add trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Research Topic`
3. Add two fields:

| Name | Value |
|------|-------|
| `topic` | `When was the Eiffel Tower built and how tall is it in feet?` |
| `recipient` | `team@example.com` |

### Step 3: Add the AI Agent

1. Add **AI Agent** → rename to `Research Agent`
2. Set **Prompt** (Expression mode):
   ```
   Research this topic and provide a brief summary (3-4 sentences):
   
   {{ $json.topic }}
   
   Use Wikipedia for facts. Use Calculator for any unit conversions.
   ```
3. Set **System Message**:
   ```
   You are a research assistant. Your job is to find accurate information.
   
   Rules:
   - Use Wikipedia to look up facts. Do NOT guess.
   - Use Calculator for any math or conversions.
   - Call each tool at most twice. If you still can't find the answer, explain what you tried.
   - Keep your summary brief: 3-4 sentences maximum.
   - If you can't find information, say so clearly.
   ```

### Step 4: Connect sub-nodes to the Agent

1. Add **OpenRouter Chat Model** (or your preferred provider) → connect to Agent
2. Add **Wikipedia** tool → connect to Agent
3. Add **Calculator** tool → connect to Agent

### Step 5: Format the email draft

1. Add **Edit Fields** → rename to `Format Email Draft`
2. Add fields:

| Name | Value (Expression) |
|------|--------------------|
| `to` | `{{ $node['Input — Research Topic'].json.recipient }}` |
| `subject` | `Research Summary: {{ $node['Input — Research Topic'].json.topic.substring(0, 50) }}...` |
| `body` | `{{ $json.output }}` |
| `drafted_at` | `{{ $now.format('yyyy-MM-dd HH:mm') }}` |
| `status` | `pending_approval` |

### Step 6: Add the approval gate

1. Add **Wait** → rename to `Wait — Approval`
2. Set **Resume**: `On Webhook Call`
3. In Options, set **Webhook Suffix**: `{{ $execution.id }}`

### Step 7: Add the output

1. Add **Edit Fields** → rename to `Output — Approved`
2. Add fields:

| Name | Value (Expression) |
|------|--------------------|
| `email_to` | `{{ $node['Format Email Draft'].json.to }}` |
| `email_subject` | `{{ $node['Format Email Draft'].json.subject }}` |
| `email_body` | `{{ $node['Format Email Draft'].json.body }}` |
| `status` | `approved` |
| `approved_at` | `{{ $now.format('yyyy-MM-dd HH:mm') }}` |

3. Enable **Keep Only Set**

### Step 8: Test

1. Click **Execute Workflow**
2. Watch the agent use Wikipedia and Calculator
3. See the draft email at "Format Email Draft"
4. Copy the **Test URL** from the Wait node
5. Open the URL in your browser → workflow completes

::::

### What's in the Workflow

| Node | What it does |
|------|-------------|
| **Manual Trigger** | Starts the workflow |
| **Input — Research Topic** | Provides topic and recipient |
| **Research Agent** | Uses Wikipedia and Calculator to find facts |
| **Format Email Draft** | Structures output as an email |
| **Wait — Approval** | Pauses until human approves |
| **Output — Approved** | Final output ready for sending |

### Test It

| Topic | What happens |
|-------|-------------|
| `When was the Eiffel Tower built and how tall is it in feet?` | Wikipedia + Calculator |
| `What is the population of Japan?` | Wikipedia only |
| `What is 15% of 2,340?` | Calculator only |

### The Key Pattern

```
User input → Agent researches (read-only) → Draft created → WAIT → Human approves → Ready to send
```

The agent does all the research freely because the tools are read-only. The email action (write) is blocked until a human reviews the draft. In production, you'd connect Gmail or Outlook after the Wait node.

---

## Debugging Tool Calls

When tools don't work as expected, check these in order:

| Problem | What to check | Fix |
|---------|---------------|-----|
| Agent doesn't use any tool | Is the description clear? | Add "Use this tool when..." |
| Agent uses wrong tool | Are descriptions overlapping? | Add "Do NOT use when..." |
| Agent calls tool repeatedly | Is it getting useful results? | Check tool output, improve query |
| Agent ignores tool output | Is system message too restrictive? | Adjust instructions |
| Tool returns error | Are credentials set up? | Check Settings → Credentials |

### Debug Checklist

1. **Click the Agent node** → look at the "Steps" in the output panel
2. **Check which tools were called** and with what inputs
3. **Read the tool outputs** — did they return useful data?
4. **Look at the final answer** — did the agent use the tool output?

---

## MCP: Connecting to External Tool Servers (Optional)

**MCP** (Model Context Protocol) is a standard for connecting AI agents to external tools. Instead of building tools inside n8n, you can connect to tools hosted elsewhere.

n8n has native MCP support with three nodes:

| Node | What it does |
|------|-------------|
| **MCP Client Tool** | Connect your AI Agent to tools from an external MCP server |
| **MCP Client** | Use MCP tools in regular workflows (not agents) |
| **MCP Server Trigger** | Expose your n8n workflows as tools for other MCP clients |

### When to Use MCP

- You want to use tools from services like Notion, Slack, or custom APIs that provide MCP endpoints
- You want to expose your n8n workflows as tools for Claude Desktop, Cursor, or other MCP clients
- You're building a system where multiple AI applications share the same tools

### Example: Connecting to an MCP Server

1. Add **MCP Client Tool** as a sub-node to your AI Agent
2. Configure the **SSE Endpoint** URL of the MCP server
3. Set authentication if required
4. Choose which tools to expose to the agent (All or Selected)

The agent can now use tools from that external server.

```{note}
MCP is powerful but adds complexity. For this course, the built-in tools (Wikipedia, Calculator, HTTP Request) are sufficient. Explore MCP when you need tools that aren't available natively in n8n.

**Learn more:** [n8n MCP Documentation](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolmcp/)
```

---

## Summary

| Concept | Key Point |
|---------|----------|
| **Tool calling** | Model requests tools; n8n executes them |
| **ReAct loop** | Reason → Call Tool → Observe → Repeat or Answer |
| **Read-only tools** | Safe to use freely (Wikipedia, Calculator) |
| **Write tools** | Need human approval (send email, create ticket) |
| **Descriptions** | "Use when... / Do NOT use when..." format |
| **Guardrails** | Max iterations, system message rules, approval gates |
| **MCP** | Standard for connecting to external tool servers |

---

## Next Steps

The **Guardrails & Safety** chapter goes deeper into safety patterns, including input validation, output filtering, and comprehensive safety checklists for production agents.

The **Node Toolbox** appendix provides reference documentation for all nodes covered in this course.